# How to run expected coverage

Expected coverage provides a simple and interpretable tool to diagnose issues in the posterior. In comparison to other diagnostic tools such as L-C2ST, it requires relatively few additional simulations (~200) and it does not rely on any additional hyperparameters (as TARP would) or additional neural network training.

Expected coverage allows you to evaluate whether your posterior is, on average across many observations (prior predictive samples) is over- or under-confident.

You can run expected coverage with the `sbi` toolbox as shown below:

## Main syntax

```python
from sbi.diagnostics import run_sbc
from sbi.analysis.plot import sbc_rank_plot

# Obtain your `posterior_estimator` with NPE, NLE, NRE.
posterior_estimator = DirectPosterior(posterior_net, prior)

num_sbc_samples = 200  # choose a number of sbc runs, should be ~100s
prior_samples = prior.sample((num_sbc_samples,))
prior_predictives = simulate(prior_samples)

# run SBC: for each inference we draw 1000 posterior samples.
num_posterior_samples = 1_000
ranks, dap_samples = run_sbc(
    prior_samples,
    prior_predictives,
    posterior_estimator,
    reduce_fns=lambda theta, x: -posterior_estimator.log_prob(theta, x),
    num_posterior_samples=num_posterior_samples,
    use_batched_sampling=False,  # `True` can give speed-ups, but can cause memory issues.
)
fig, ax = sbc_rank_plot(
    ranks,
    num_posterior_samples,
    plot_type="cdf",
    num_bins=20,
    figsize=(5, 3),
)
```

This will return a figure such as the following:

<img src="sbc_rank_plot.png" width="500">

You can interpret this plots as follows:
- If the blue line is below the diagonal, then the `posterior_estimator` is (on average) over
-confident.
- If the line is above the gray region, then the `posterior_esitmator` is, on average, under-confident.
- If the line is within the gray region, then we cannot reject the null hypothesis that the posterior is well-calibrated.